In [1]:
import pandas as pd
import tabula

In [2]:
filename = 'Freshco_2'
raw_df = tabula.read_pdf(filename+'.pdf',multiple_tables=True,pages='all')

In [9]:
raw_df

[                                        0
 0  Phone # (02) 9737 9969, (02) 9737 9979
 1                    Fax # (02) 9737 9989
 2    E-mail: accounts@freshcofoods.com.au
 3        Website: www.freshcofoods.com.au,
                          0           1              2
 0     ABN # 52 141 913 171         NaN            NaN
 1                      NaN        Date  Tax Invoice #
 2  NIT 16, 2 SLOUGH AVENUE         NaN            NaN
 3    SILVERWATER, NSW 2128  22/02/2017           9109,
               0      1    2
 0  Customer ABN  Terms  Rep,
                                                     0      1   2  \
 0                                Description Qty Unit  Price NaN   
 1                      BOMBAY BIRYANI 60G SHAN 12 PKT   1.05 NaN   
 2                     FISH SEASONNING 50G SHAN 12 PKT   1.05 NaN   
 3                       CHICKEN CURRY 50G SHAN 12 PKT   1.05 NaN   
 4                         SEEKH KEBAB 50G SHAN 12 PKT   1.05 NaN   
 5                        BEEF BIRY

In [6]:
num_of_pages = len(raw_df)
num_of_pages

4

In [7]:
def isPresent(df,str):
    result_ser = df.iloc[:,0].str.contains(str)
    result_list = result_ser.tolist()
    for i in result_list:
        if i == True:
            return True
    return False

In [8]:
description = []
Qty = []
Unit = []
Price = []
Gst = []
Gst_Amt = []
Amount = []

gst_and_gstamt = []
dqu = []

total = []

In [ ]:
starting_str = 'Description'
ending_str = 'EFT PAYMENT DETAILS'
for index in range(0,num_of_pages):
    df = raw_df[index]
    result = isPresent(df,starting_str)
    if result == True:
        start_index = df.index[df.iloc[:,0].str.contains(starting_str) == True]
        sub_result = isPresent(df,'NOTE')
        if sub_result == True:
            end_index   = df.index[df.iloc[:,0].str.contains('NOTE') == True]
        else:
            end_index   = df.index[df.iloc[:,0].str.contains(ending_str) == True]
        table_df = df[start_index[0]+1:end_index[0]]
        Price = table_df.iloc[:,1].tolist()
        Amount = table_df.iloc[:,5].tolist()
        gst_and_gstamt = table_df.iloc[:,3].tolist()
        dqu = table_df.iloc[:,0].tolist()
        
        for item in gst_and_gstamt:
            j=item.split(" ")
            Gst.append(j[0])
            Gst_Amt.append(j[1])
            
        for item in dqu:
            j=item.split(" ")
            Unit.append(j[-1])
            Qty.append(j[-2])
            description.append(j[:-2])
        
        index_of_total = df.index[df.iloc[:,3].str.contains('Total') == True]
        total = df.iloc[index_of_total,5].tolist()

In [ ]:
data_dic = {'Description':description,'Qty':Qty,'Unit':Unit,"Price":Price,"GST":Gst,'GST AMT':Gst_Amt,'Amount':Amount}


In [ ]:
dataframe_result = pd.DataFrame(data=data_dic)

index = ['Description', 'Qty', 'Unit', 'Price','GST','GST AMT','Amount']

dataframe_result = dataframe_result.reindex(index,axis=1);

dataframe_result = dataframe_result.append(pd.Series(['','','','','','','Total: '+total[0]], index=dataframe_result.columns), ignore_index=True)

dataframe_result.to_csv(filename+'.csv',index=None,header=True)